# Analyse a picture for project move color prototype

## Import and variables

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
#%matplotlib inline
%matplotlib tk

In [ ]:
DIST = 2
BORDER_OF_PREDICTION = 0.5

In [ ]:
PLT_RED = 'r+'
PLT_BLUE = 'bo'
PLT_CYAN = 'cs'

## Functions

In [ ]:
def prediction(combined_squared_error):
    return (1 / (1 + combined_squared_error))

In [ ]:
def error_function(hsv_pixel):
    error_hue = hue - hsv_pixel[0, 0, 0]
    error_sat = sat - hsv_pixel[0, 0, 1]
    error_val = val - hsv_pixel[0, 0, 2]

    combined_squared_error = (
        error_hue**2 * factor_hue
        + error_sat**2 * factor_sat 
        + error_val**2 * factor_val
    )
    predicted_value = prediction(combined_squared_error)
    return combined_squared_error, predicted_value

In [ ]:
def draw_pixel_value(event, x, y, flags, param):
    global clicked
    global marked_good_pixels
    global marked_bad_pixels

    if event == cv2.EVENT_LBUTTONDOWN and not clicked:
        clicked = True
        pixel = image[y, x]
        pixel = np.uint8([[pixel]])
        hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
        squared_error_and_prediction = error_function(hsv_pixel)
        draw_function((x,y), squared_error_and_prediction, hsv_pixel)
        marked_good_pixels.append(hsv_pixel[0, 0, :])
        
    elif event == cv2.EVENT_RBUTTONDOWN and not clicked:
        clicked = True
        pixel = image[y, x]
        pixel = np.uint8([[pixel]])
        hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
        squared_error_and_prediction = error_function(hsv_pixel)
        draw_function((x,y), squared_error_and_prediction, hsv_pixel)
        marked_bad_pixels.append(hsv_pixel[0 , 0, :])
        
    elif event == cv2.EVENT_LBUTTONUP or cv2.EVENT_RBUTTONUP:
        clicked = False

In [ ]:
def draw_function(point_in_image, calculated_point, hsv_pixel):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    circle_size = rgb_image.shape[0] * 10 // 640
    text_size = rgb_image.shape[0] * 4 // 640

    cv2.circle(rgb_image, point_in_image, 2*circle_size, [0,0,0], thickness=-1)
    cv2.circle(rgb_image, point_in_image, circle_size, [255,255,255], thickness=-1)
    cv2.putText(rgb_image, f'{point_in_image}', (10, rgb_image.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX, text_size, (255,255,255))
    
    x_min = np.min([calculated_point[0]-2, -10])
    x_max = np.max([calculated_point[0]+2, 10])
    x_step = np.round((x_max - x_min) / 100, decimals=2)
    x_func = np.arange(x_min, x_max, x_step)
    y_func = list(map(prediction, x_func**2))

    fig, axes = plt.subplot_mosaic([
        ['image', 'graph'],
        ['pixel', 'graph']
    ])
    
    fig.set_dpi(200)    
    axes['image'].imshow(rgb_image)
    
    axes['graph'].plot(x_func, y_func)
    axes['graph'].plot([x_min, calculated_point[0], calculated_point[0]], [calculated_point[1], calculated_point[1], 0], 'r--')
    axes['graph'].plot(calculated_point[0], calculated_point[1], 'ro')

    pixel = cv2.cvtColor(hsv_pixel, cv2.COLOR_HSV2RGB)
    pixel_example = np.zeros((100, 100, 3), dtype=np.uint8)
    pixel_example[:, :, 0] = pixel[0, 0, 0]
    pixel_example[:, :, 1] = pixel[0, 0, 1]
    pixel_example[:, :, 2] = pixel[0, 0, 2]
    axes['pixel'].imshow(pixel_example)
    
    fig.suptitle(f'HSV:{hsv_pixel[0, 0, :]} result: {calculated_point[1]:.5f}')
    plt.show()

In [ ]:
def click_and_evaluate_pixel(image):
    title = 'click at pixel'
    cv2.namedWindow(title)
    cv2.setMouseCallback(title, draw_pixel_value)
    
    while True:
        cv2.imshow(title, image)
        if cv2.waitKey(30) == 27:
            break
    
    cv2.destroyAllWindows()

In [ ]:
def get_roi(image, description=''):
    x, y, w, h = cv2.selectROI(f'select {description}', image, False, False)
    cv2.destroyAllWindows()
    
    print(x, y, w, h)
    
    x_lower = x
    x_upper = x + w
    y_lower = y
    y_upper = y + h
    
    roi = image[y_lower:y_upper, x_lower:x_upper]
    roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    plt.figure()
    plt.title(description)
    plt.imshow(roi)
    plt.show()
    return x_lower, x_upper, y_lower, y_upper

## Load images and settings

In [ ]:
image_filename = '../build/debug_image.jpg'

In [ ]:
image = cv2.imread(image_filename)
hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_image)

In [ ]:
settings_filename = '../build/debug_hsv.txt'
settings = [''] * 6

with open(settings_filename, 'r') as fid:
    for i in range(len(settings)):
        settings[i] = fid.readline().strip()

hue = float(settings[0])
sat = float(settings[1])
val = float(settings[2])
factor_hue = float(settings[3])
factor_sat = float(settings[4])
factor_val = float(settings[5])

print(hue)
print(sat)
print(val)
print(factor_hue)
print(factor_sat)
print(factor_val)

## Image validation

In [ ]:
for col in range(0, hsv_image.shape[0], DIST):
    for row in range(0, hsv_image.shape[1], DIST):
        hsv_pixel = np.uint8([[hsv_image[col, row, :]]])
        #print(pixel)
        squared_error_and_prediction = error_function(hsv_pixel)
        if squared_error_and_prediction[1] >= BORDER_OF_PREDICTION:
            rgb_image[col, row, :] = (0, 0, 255)
        else:
            rgb_image[col, row, :] = (255, 0, 0)

In [ ]:
plt.imshow(rgb_image)

In [ ]:
colored_bgr_image = cv2.cvtColor(rgb_image, cv2.COLOR_RGB2BGR)

In [ ]:
cv2.imshow('calibration result', colored_bgr_image)
cv2.waitKey(0)
cv2.destroyAllWindows()

## Histograms and plots

In [ ]:
x_lower, x_upper, y_lower, y_upper = get_roi(image, 'only positives')
neg_x_lower, neg_x_upper, neg_y_lower, neg_y_upper = get_roi(image, 'all positives')

good_values = []
bad_values = []
for col in range(0, hsv_image.shape[0], DIST):
    for row in range(0, hsv_image.shape[1], DIST):
        if y_lower < col < y_upper:
            if x_lower < row < x_upper:
                good_values.append(hsv_image[col, row, :])
        if neg_y_lower > col or col > neg_y_upper:
            bad_values.append(hsv_image[col, row, :])
        elif neg_x_lower > row or row > neg_x_upper:
            bad_values.append(hsv_image[col, row, :])

good_values = np.array(good_values)
bad_values = np.array(bad_values)

good_hue = good_values[:, 0]
good_sat = good_values[:, 1]
good_val = good_values[:, 2]

bad_hue = bad_values[:, 0]
bad_sat = bad_values[:, 1]
bad_val = bad_values[:, 2]

fig, axes = plt.subplots(1, 3)
fig.suptitle('good pixels')
_ = axes[0].hist(good_hue, bins=36, range=(0, 180))
_ = axes[0].set_xlabel('hue')
_ = axes[1].hist(good_sat, bins=51, range=(0, 255))
_ = axes[1].set_xlabel('sat')
_ = axes[2].hist(good_val, bins=51, range=(0, 255))
_ = axes[2].set_xlabel('val')

fig, axes = plt.subplots(1, 3)
fig.suptitle('bad values')
_ = axes[0].hist(bad_hue, bins=36, range=(0, 180), color='red')
_ = axes[0].set_xlabel('hue')
_ = axes[1].hist(bad_sat, bins=51, range=(0, 255), color='red')
_ = axes[1].set_xlabel('sat')
_ = axes[2].hist(bad_val, bins=51, range=(0, 255), color='red')
_ = axes[2].set_xlabel('val')

In [ ]:
offset = 10

min_zoom_plot_hue = min(good_hue) - offset
max_zoom_plot_hue = max(good_hue) + offset
min_zoom_plot_sat = min(good_sat) - offset
max_zoom_plot_sat = max(good_sat) + offset
min_zoom_plot_val = min(good_val) - offset
max_zoom_plot_val = max(good_val) + offset

fig = plt.figure(layout='constrained')
fig.suptitle('position comparison')
axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].plot(hue, sat, PLT_CYAN)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')

_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].plot(val, sat, PLT_CYAN)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].plot(hue, val, PLT_CYAN)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_ylim([255+offset, -offset])


fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].plot(hue, sat, PLT_CYAN)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].plot(val, sat, PLT_CYAN)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].plot(hue, val, PLT_CYAN)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((max_zoom_plot_val, min_zoom_plot_val))

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison without positives')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(hue, sat, PLT_CYAN)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(val, sat, PLT_CYAN)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(hue, val, PLT_CYAN)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((max_zoom_plot_val, min_zoom_plot_val))

plt.show()

## Show valid range

In [ ]:
offset = 10

X = np.arange(0, 180)
Y = np.arange(0, 256)
Z = np.arange(0, 256)

good_decision_hue = []
good_decision_sat = []
good_decision_val = []
bad_decision_hue = []
bad_decision_sat = []
bad_decision_val = []

def help_func(tmp_hue, tmp_sat, tmp_val):
    hsv_pixel = np.uint8([[[0, tmp_sat, tmp_val]]])
    _, prediction_ = error_function(hsv_pixel)
    return prediction_

for i in X:
    for j in Y:
        for k in Z:
            ans = help_func(i, j, k)
            if ans > 0.5:
                good_decision_hue.append(i)
                good_decision_sat.append(j)
                good_decision_val.append(k)
            else:
                bad_decision_hue.append(i)
                bad_decision_sat.append(j)
                bad_decision_val.append(k)

fig = plt.figure(layout='constrained')
fig.suptitle('possible positives compared to negatives')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_decision_hue, bad_decision_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_decision_hue, good_decision_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
                                 
_ = axes['val_and_sat'].plot(bad_decision_val, bad_decision_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_decision_val, good_decision_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')

_ = axes['hue_and_val'].plot(bad_decision_hue, bad_decision_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_decision_hue, good_decision_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_ylim([255+offset, -offset])


plt.show()

## Click and evaluate

## optional
hue = float(settings[0])
sat = float(settings[1])
val = float(settings[2])
factor_hue = float(0)
factor_sat = float(0)
factor_val = float(settings[5])

In [ ]:
clicked = False
marked_good_pixels = []
marked_bad_pixels = []

# left click good, right click bad
click_and_evaluate_pixel(image)

marked_good_pixels = np.array(marked_good_pixels)
marked_bad_pixels = np.array(marked_bad_pixels)

single_example_size = 200

if marked_good_pixels.size > 0:
    good_color_example = np.ones((single_example_size, len(marked_good_pixels)*single_example_size, 3), dtype=np.uint8)
    
    for pos in range(len(marked_good_pixels)):
        good_color_example[:, pos * single_example_size : (pos+1) * single_example_size, :] = good_color_example[0, pos * single_example_size, :] * marked_good_pixels[pos]
        
        if pos != 0:
            good_color_example[:, pos * single_example_size, :] = 0
        
    good_color_example = cv2.cvtColor(good_color_example, cv2.COLOR_HSV2RGB)
    
    for pos in range(len(marked_good_pixels)):
        prediction_ = error_function(np.uint8([[good_color_example[0, pos*single_example_size, :]]]))[1]
        cv2.putText(good_color_example, f'{prediction_:0.4f}', (pos*single_example_size, good_color_example.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    plt.figure()
    plt.title('good colors')
    plt.imshow(good_color_example)
    plt.show()

if marked_bad_pixels.size > 0:

    bad_color_example = np.ones((single_example_size, len(marked_bad_pixels)*single_example_size, 3), dtype=np.uint8)
    
    for pos in range(len(marked_bad_pixels)):
        bad_color_example[:, pos * single_example_size : (pos+1) * single_example_size, :] = bad_color_example[0, pos * single_example_size, :] * marked_bad_pixels[pos]
        if pos != 0:
            bad_color_example[:, pos * single_example_size, :] = 0
        
    bad_color_example = cv2.cvtColor(bad_color_example, cv2.COLOR_HSV2RGB)

    for pos in range(len(marked_bad_pixels)):
        predicton_ = error_function(np.uint8([[bad_color_example[0, pos*single_example_size, :]]]))[1]
        cv2.putText(bad_color_example, f'{prediction_:0.4f}', (pos*single_example_size, bad_color_example.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

    plt.figure()
    plt.title('bad colors')
    plt.imshow(bad_color_example)
    plt.show()

In [ ]:
good_hue = marked_good_pixels[:, 0]
good_sat = marked_good_pixels[:, 1]
good_val = marked_good_pixels[:, 2]

bad_hue = marked_bad_pixels[:, 0]
bad_sat = marked_bad_pixels[:, 1]
bad_val = marked_bad_pixels[:, 2]

offset = 10

min_zoom_plot_hue = min(good_hue) - offset
max_zoom_plot_hue = max(good_hue) + offset
min_zoom_plot_sat = min(good_sat) - offset
max_zoom_plot_sat = max(good_sat) + offset
min_zoom_plot_val = min(good_val) - offset
max_zoom_plot_val = max(good_val) + offset

fig = plt.figure(layout='constrained')
fig.suptitle('position comparison')
axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_ylim([255+offset, -offset])

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].plot(good_hue, good_sat, PLT_BLUE)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].plot(good_val, good_sat, PLT_BLUE)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].plot(good_hue, good_val, PLT_BLUE)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((max_zoom_plot_val, min_zoom_plot_val))

fig = plt.figure(layout='constrained')
fig.suptitle('zoomed comparison without positives')

axes = fig.subplot_mosaic([
        ['hue_and_sat', 'val_and_sat'],
        ['hue_and_val', 'none']
    ], 
    empty_sentinel='none'
)

_ = axes['hue_and_sat'].plot(bad_hue, bad_sat, PLT_RED)
_ = axes['hue_and_sat'].grid()
_ = axes['hue_and_sat'].set_xlabel('hue')
_ = axes['hue_and_sat'].set_ylabel('sat')
_ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                 
_ = axes['val_and_sat'].plot(bad_val, bad_sat, PLT_RED)
_ = axes['val_and_sat'].grid()
_ = axes['val_and_sat'].set_xlabel('val')
_ = axes['val_and_sat'].set_ylabel('sat')
_ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
_ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))

_ = axes['hue_and_val'].plot(bad_hue, bad_val, PLT_RED)
_ = axes['hue_and_val'].grid()
_ = axes['hue_and_val'].set_xlabel('hue')
_ = axes['hue_and_val'].set_ylabel('val')
_ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
_ = axes['hue_and_val'].set_ylim((max_zoom_plot_val, min_zoom_plot_val))

## Video comparison

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

class VideoComparison:
    def __init__(self, hue, sat, val, factor_hue, factor_sat, factor_val):
        self.PLT_RED = 'r+'
        self.PLT_BLUE = 'bo'
        self.PLT_CYAN = 'cs'
        
        self.clicked = False
        self.marked_good_pixels = []
        self.marked_bad_pixels = []
        self.image = None
        self.ret = None
        self.single_example_size = 200
        

        self.hue = hue
        self.sat = sat
        self.val = val
        self.factor_hue = factor_hue
        self.factor_sat = factor_sat
        self.factor_val = factor_val

    
    def evaluate_from_camera(self, cam_nbr):
        title = 'click at pixel'
    
        cap = cv2.VideoCapture(cam_nbr)
    
        cv2.namedWindow(title)
        cv2.setMouseCallback(title, self.draw_pixel_value)
        
        while cap.isOpened():
            self.ret, self.image = cap.read()
            if not self.ret:
                break
            cv2.imshow(title, self.image)
            if cv2.waitKey(30) == 27:
                break
        cap.release()
        cv2.destroyAllWindows()
        return False

    
    def draw_pixel_value(self, event, x, y, flags, param): 
        #global clicked global marked_good_pixels global marked_bad_pixels
        if event == cv2.EVENT_LBUTTONDOWN and not self.clicked:
            self.clicked = True
            if self.ret:
                pixel = self.image[y, x]
                pixel = np.uint8([[pixel]])
                hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
                
                squared_error_and_prediction = self.error_function(hsv_pixel)
                self.draw_function((x,y), squared_error_and_prediction, hsv_pixel)
                self.marked_good_pixels.append(hsv_pixel[0, 0, :])
                
            else:
                print('no image available')
                
        elif event == cv2.EVENT_RBUTTONDOWN and not self.clicked:
            clicked = True
            if self.ret:
                pixel = self.image[y, x]
                pixel = np.uint8([[pixel]])
                hsv_pixel = cv2.cvtColor(pixel, cv2.COLOR_BGR2HSV)
                
                squared_error_and_prediction = self.error_function(hsv_pixel)
                self.draw_function((x,y), squared_error_and_prediction, hsv_pixel)
                self.marked_bad_pixels.append(hsv_pixel[0 , 0, :])

            else:
                print('no image available')
                
        elif event == cv2.EVENT_LBUTTONUP or cv2.EVENT_RBUTTONUP:
            self.clicked = False

            
    def error_function(self, hsv_pixel):
        error_hue = self.hue - hsv_pixel[0, 0, 0]
        error_sat = self.sat - hsv_pixel[0, 0, 1]
        error_val = self.val - hsv_pixel[0, 0, 2]
    
        combined_squared_error = (
            error_hue**2 * self.factor_hue
            + error_sat**2 * self.factor_sat 
            + error_val**2 * self.factor_val
        )
        
        predicted_value = self.prediction(combined_squared_error)
        return combined_squared_error, predicted_value

        
    def draw_function(self, point_in_image, calculated_point, hsv_pixel):
        rgb_image = cv2.cvtColor(self.image, cv2.COLOR_BGR2RGB)
        circle_size = rgb_image.shape[0] * 10 // 640
        text_size = rgb_image.shape[0] * 4 // 640
    
        cv2.circle(rgb_image, point_in_image, 2*circle_size, [0,0,0], thickness=-1)
        cv2.circle(rgb_image, point_in_image, circle_size, [255,255,255], thickness=-1)
        cv2.putText(rgb_image, f'{point_in_image}', (10, rgb_image.shape[0]-10), cv2.FONT_HERSHEY_SIMPLEX, text_size, (255,255,255))
        
        x_min = np.min([calculated_point[0]-2, -10])
        x_max = np.max([calculated_point[0]+2, 10])
        x_step = np.round((x_max - x_min) / 100, decimals=2)
        x_func = np.arange(x_min, x_max, x_step)
        y_func = list(map(self.prediction, x_func**2))
    
        fig, axes = plt.subplot_mosaic([
            ['image', 'graph'],
            ['pixel', 'graph']
        ])
        
        fig.set_dpi(200)        
        axes['image'].imshow(rgb_image)
        
        axes['graph'].plot(x_func, y_func)
        axes['graph'].plot([x_min, calculated_point[0], calculated_point[0]], [calculated_point[1], calculated_point[1], 0], 'r--')
        axes['graph'].plot(calculated_point[0], calculated_point[1], 'ro')
    
        pixel = cv2.cvtColor(hsv_pixel, cv2.COLOR_HSV2RGB)
        pixel_example = np.zeros((100, 100, 3), dtype=np.uint8)
        pixel_example[:, :, 0] = pixel[0, 0, 0]
        pixel_example[:, :, 1] = pixel[0, 0, 1]
        pixel_example[:, :, 2] = pixel[0, 0, 2]
    
        axes['pixel'].imshow(pixel_example)
        
        fig.suptitle(f'HSV:{hsv_pixel[0, 0, :]} result: {calculated_point[1]:.5f}')
        plt.show()

    
    def prediction(self, combined_squared_error):
        return (1 / (1 + combined_squared_error))

    
    def show_result(self):
        self.marked_good_pixels = np.array(self.marked_good_pixels)
        self.marked_bad_pixels = np.array(self.marked_bad_pixels)
        
        if self.marked_good_pixels.size > 0:
            good_color_example = np.ones((self.single_example_size, len(self.marked_good_pixels)*self.single_example_size, 3), dtype=np.uint8)
            
            for pos in range(len(self.marked_good_pixels)):
                good_color_example[:, pos * self.single_example_size : (pos+1) * self.single_example_size, :] = good_color_example[0, pos * self.single_example_size, :] * self.marked_good_pixels[pos]
                
                if pos != 0:
                    good_color_example[:, pos * self.single_example_size, :] = 0
                
            good_color_example = cv2.cvtColor(good_color_example, cv2.COLOR_HSV2RGB)
            
            for pos in range(len(self.marked_good_pixels)):
                prediction_ = self.error_function(np.uint8([[good_color_example[0, pos*self.single_example_size, :]]]))[1]
                cv2.putText(good_color_example, f'{prediction_:0.4f}', (pos*self.single_example_size, good_color_example.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))
            
            plt.figure()
            plt.title('good colors')
            plt.imshow(good_color_example)
            plt.show()
        
        if self.marked_bad_pixels.size > 0:
        
            bad_color_example = np.ones((self.single_example_size, len(self.marked_bad_pixels)*self.single_example_size, 3), dtype=np.uint8)
            
            for pos in range(len(self.marked_bad_pixels)):
                bad_color_example[:, pos * self.single_example_size : (pos+1) * self.single_example_size, :] = bad_color_example[0, pos * self.single_example_size, :] * self.marked_bad_pixels[pos]
                if pos != 0:
                    bad_color_example[:, pos * self.single_example_size, :] = 0
                
            bad_color_example = cv2.cvtColor(bad_color_example, cv2.COLOR_HSV2RGB)
        
            for pos in range(len(self.marked_bad_pixels)):
                predicton_ = self.error_function(np.uint8([[bad_color_example[0, pos*self.single_example_size, :]]]))[1]
                cv2.putText(bad_color_example, f'{prediction_:0.4f}', (pos*self.single_example_size, bad_color_example.shape[0]-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255))

            plt.figure()
            plt.title('bad colors')
            plt.imshow(bad_color_example)
            plt.show()


    def draw_statistics(self):
        good_hue = self.marked_good_pixels[:, 0]
        good_sat = self.marked_good_pixels[:, 1]
        good_val = self.marked_good_pixels[:, 2]
        
        bad_hue = self.marked_bad_pixels[:, 0]
        bad_sat = self.marked_bad_pixels[:, 1]
        bad_val = self.marked_bad_pixels[:, 2]
        
        offset = 10
        
        min_zoom_plot_hue = min(good_hue) - offset
        max_zoom_plot_hue = max(good_hue) + offset
        min_zoom_plot_sat = min(good_sat) - offset
        max_zoom_plot_sat = max(good_sat) + offset
        min_zoom_plot_val = min(good_val) - offset
        max_zoom_plot_val = max(good_val) + offset
        
        fig = plt.figure(layout='constrained')
        fig.suptitle('position comparison')
        axes = fig.subplot_mosaic([
                ['hue_and_sat', 'val_and_sat'],
                ['hue_and_val', 'none']
            ], 
            empty_sentinel='none'
        )
        
        _ = axes['hue_and_sat'].plot(bad_hue, bad_sat, self.PLT_RED)
        _ = axes['hue_and_sat'].plot(good_hue, good_sat, self.PLT_BLUE)
        _ = axes['hue_and_sat'].grid()
        _ = axes['hue_and_sat'].set_xlabel('hue')
        _ = axes['hue_and_sat'].set_ylabel('sat')
        _ = axes['val_and_sat'].plot(bad_val, bad_sat, self.PLT_RED)
        _ = axes['val_and_sat'].plot(good_val, good_sat, self.PLT_BLUE)
        _ = axes['val_and_sat'].grid()
        _ = axes['val_and_sat'].set_xlabel('val')
        _ = axes['val_and_sat'].set_ylabel('sat')
        _ = axes['hue_and_val'].plot(bad_hue, bad_val, self.PLT_RED)
        _ = axes['hue_and_val'].plot(good_hue, good_val, self.PLT_BLUE)
        _ = axes['hue_and_val'].grid()
        _ = axes['hue_and_val'].set_xlabel('hue')
        _ = axes['hue_and_val'].set_ylabel('val')
        
        fig = plt.figure(layout='constrained')
        fig.suptitle('zoomed comparison')
        
        axes = fig.subplot_mosaic([
                ['hue_and_sat', 'val_and_sat'],
                ['hue_and_val', 'none']
            ], 
            empty_sentinel='none'
        )
        
        _ = axes['hue_and_sat'].plot(bad_hue, bad_sat, self.PLT_RED)
        _ = axes['hue_and_sat'].plot(good_hue, good_sat, self.PLT_BLUE)
        _ = axes['hue_and_sat'].grid()
        _ = axes['hue_and_sat'].set_xlabel('hue')
        _ = axes['hue_and_sat'].set_ylabel('sat')
        _ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
        _ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                         
        _ = axes['val_and_sat'].plot(bad_val, bad_sat, self.PLT_RED)
        _ = axes['val_and_sat'].plot(good_val, good_sat, self.PLT_BLUE)
        _ = axes['val_and_sat'].grid()
        _ = axes['val_and_sat'].set_xlabel('val')
        _ = axes['val_and_sat'].set_ylabel('sat')
        _ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
        _ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
        
        _ = axes['hue_and_val'].plot(bad_hue, bad_val, self.PLT_RED)
        _ = axes['hue_and_val'].plot(good_hue, good_val, self.PLT_BLUE)
        _ = axes['hue_and_val'].grid()
        _ = axes['hue_and_val'].set_xlabel('hue')
        _ = axes['hue_and_val'].set_ylabel('val')
        _ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
        _ = axes['hue_and_val'].set_ylim((min_zoom_plot_val, max_zoom_plot_val))
        
        fig = plt.figure(layout='constrained')
        fig.suptitle('zoomed comparison without positives')
        
        axes = fig.subplot_mosaic([
                ['hue_and_sat', 'val_and_sat'],
                ['hue_and_val', 'none']
            ], 
            empty_sentinel='none'
        )
        
        _ = axes['hue_and_sat'].plot(bad_hue, bad_sat, self.PLT_RED)
        _ = axes['hue_and_sat'].grid()
        _ = axes['hue_and_sat'].set_xlabel('hue')
        _ = axes['hue_and_sat'].set_ylabel('sat')
        _ = axes['hue_and_sat'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
        _ = axes['hue_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
                                         
        _ = axes['val_and_sat'].plot(bad_val, bad_sat, self.PLT_RED)
        _ = axes['val_and_sat'].grid()
        _ = axes['val_and_sat'].set_xlabel('val')
        _ = axes['val_and_sat'].set_ylabel('sat')
        _ = axes['val_and_sat'].set_xlim((min_zoom_plot_val, max_zoom_plot_val))
        _ = axes['val_and_sat'].set_ylim((min_zoom_plot_sat, max_zoom_plot_sat))
        
        _ = axes['hue_and_val'].plot(bad_hue, bad_val, self.PLT_RED)
        _ = axes['hue_and_val'].grid()
        _ = axes['hue_and_val'].set_xlabel('hue')
        _ = axes['hue_and_val'].set_ylabel('val')
        _ = axes['hue_and_val'].set_xlim((min_zoom_plot_hue, max_zoom_plot_hue))
        _ = axes['hue_and_val'].set_ylim((min_zoom_plot_val, max_zoom_plot_val))

In [ ]:
obj = VideoComparison(hue, sat, val, factor_hue, factor_sat, factor_val)
if obj.evaluate_from_camera(2):
    obj.show_result()
    obj.draw_statistics()